In [1]:
import numpy as np
import xarray as xr
import dask 
import os

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial' 
plt.rcParams['font.size'] = 10
plt.rcParams['pdf.fonttype'] = 42

import cartopy.crs as ccrs
import regionmask

### Preliminaries

In [2]:
###################
# Models
###################
from utils import nex_ssp_dict, cil_ssp_dict, isimip_ssp_dict, gardsv_ssp_dict, gardsv_var_dict, deepsdbc_dict

nex_models = list(nex_ssp_dict.keys())
cil_models = list(cil_ssp_dict.keys())
isi_models = list(isimip_ssp_dict.keys())
cbp_gard_models = list(gardsv_ssp_dict.keys())
cbp_gard_precip_models = [model for model in cbp_gard_models if 'pr' in gardsv_var_dict[model]]
cbp_deep_models = list(deepsdbc_dict.keys())

In [3]:
############
# Dask
############
from dask_jobqueue import PBSCluster

cluster = PBSCluster(cores=1, memory='40GB', resource_spec='pmem=40GB',
                     worker_extra_args=['#PBS -l feature=rhel7'], 
                     walltime='02:00:00')

cluster.scale(jobs=10)  # ask for jobs

from dask.distributed import Client
client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.102.201.240:45415,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Metrics time series plots

In [6]:
###############################
# Set paths
# UPDATE THIS FOR REPRODUCTION
###############################
path_in = '/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/'
path_out = '/storage/home/dcl5300/work/lafferty-sriver_inprep_tbd/figs/supplementary/'

In [9]:
def make_metric_plot(ensemble, models, ssp_dict, path_in, metric, var_id, path_out):
    out_str = path_out + ensemble + '_' + metric + '_' + var_id + '.png'
    if os.path.isfile(out_str):
        return('Already done')
    else:
        ssp_colors = {'ssp126':'#8da0cb',
                      'ssp245':'#fc8d62',
                      'ssp370':'#66c2a5',
                      'ssp585':'#e78ac3'}

        fig, axs = plt.subplots(nrows=11, ncols=4, figsize=(15,30), sharex=True)
        axs = axs.flatten()

        for idx, region in enumerate(regionmask.defined_regions.ar6.land.names[:-2]):
            key = regionmask.defined_regions.ar6.land.map_keys(region)
            axs[idx].set_title(region)
            for model in models:
                # Read netcdf or zarr
                if ensemble in ['cil-gdpcir', 'DeepSD-BC']:
                    ds = xr.open_dataset(path_in + ensemble + '/' + metric + '/' + model, engine='zarr')
                else:
                    ds = xr.open_dataset(path_in + ensemble + '/' + metric + '/' + model + '.nc')
                
                # Remove member-id
                if 'member_id' in ds.dims:
                    ds = ds.isel(member_id=0)
                
                # Mask region
                mask = regionmask.defined_regions.ar6.land.mask(ds)
            
                # Select variable
                if var_id in list(ds.data_vars):
                    ds = ds[var_id]
                    # Plot by ssp
                    for ssp in ssp_dict[model]:
                        # hard-code the one exception
                        if (ensemble == 'DeepSD-BC') and (ssp == 'ssp585') and (var_id == 'pr') and (model == 'MRI-ESM2-0'):
                            continue
                        else:
                            yrs = ds.sel(ssp=ssp).time.dt.year
                            axs[idx].plot(yrs, ds.sel(ssp=ssp).where(mask == key).mean(dim=['lat','lon']).to_numpy(), lw=1, color=ssp_colors[ssp])
                else:
                    axs[idx].text(0.1, 0.9, 'x') # mark if variable is missing

        plt.tight_layout()
        plt.savefig(out_str, dpi=300)
    
        return out_str

In [ ]:
%%time
# AVG
metric = 'avg'
delayed_res = []

delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'tas', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'tasmin', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'tasmax', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'pr', path_out))

delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'tas', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'tasmin', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'tasmax', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'pr', path_out))

delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'tas', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'tasmin', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'tasmax', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_precip_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'pr', path_out))

delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'tas', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'tasmin', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'tasmax', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'pr', path_out))

# MAX
metric = 'max'
delayed_res = []

delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'tas', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'tasmin', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'tasmax', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'pr', path_out))

delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'tas', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'tasmin', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'tasmax', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'pr', path_out))

delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'tas', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'tasmin', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'tasmax', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_precip_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'pr', path_out))

delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'tas', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'tasmin', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'tasmax', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'pr', path_out))

# MAX5d
metric = 'max5d'
delayed_res = []

delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'RX5day', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'RX5day', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_precip_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'RX5day', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'RX5day', path_out))

# DRY
metric = 'dry'
delayed_res = []

delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'count_eq_0', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'count_lt_1', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'streak_eq_0', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', nex_models, nex_ssp_dict, path_in, metric, 'streak_lt_1', path_out))

delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'count_eq_0', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'count_lt_1', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'streak_eq_0', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', cil_models, cil_ssp_dict, path_in, metric, 'streak_lt_1', path_out))

delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_precip_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'count_eq_0', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_precip_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'count_lt_1', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_precip_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'streak_eq_0', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', cbp_gard_precip_models, gardsv_ssp_dict, path_in + 'carbonplan/regridded/conservative/', metric, 'streak_lt_1', path_out))

delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'count_eq_0', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'count_lt_1', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'streak_eq_0', path_out))
delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', cbp_deep_models, deepsdbc_dict, path_in + 'carbonplan/native_grid/', metric, 'streak_lt_1', path_out))

# Compute
res = dask.compute(*delayed_res)